In [3]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Perceptron Model
class Perceptron:
    def __init__(self, input_size, learning_rate=0.1, epochs=100):
        self.weights = np.zeros(input_size + 1)  # Including bias
        self.learning_rate = learning_rate
        self.epochs = epochs

    def activation(self, x):
        return 1 if x >= 0 else 0

    def predict(self, x):
        summation = np.dot(x, self.weights[1:]) + self.weights[0]
        return self.activation(summation)

    def train(self, X, y):
        for _ in range(self.epochs):
            for inputs, label in zip(X, y):
                prediction = self.predict(inputs)
                self.weights[1:] += self.learning_rate * (label - prediction) * inputs
                self.weights[0] += self.learning_rate * (label - prediction)

# Training Perceptron
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 0, 0, 1])  # AND operation
perceptron = Perceptron(input_size=2)
perceptron.train(X, y)

print("Perceptron Predictions (AND Gate):")
for inputs in X:
    print(f"Input: {inputs}, Prediction: {perceptron.predict(inputs)}")

# 2. Multi-Layer Perceptron (MLP) with Backpropagation
class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1, epochs=10000):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs

        # Initialize weights
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_hidden = np.zeros(hidden_size)
        self.bias_output = np.zeros(output_size)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def feedforward(self, X):
        # Forward pass
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.sigmoid(self.hidden_input)
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self.sigmoid(self.final_input)
        return self.final_output

    def backpropagate(self, X, y, output):
        # Backward pass
        output_error = y - output
        output_delta = output_error * self.sigmoid_derivative(output)

        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)

        # Update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(output_delta) * self.learning_rate
        self.weights_input_hidden += X.T.dot(hidden_delta) * self.learning_rate
        self.bias_output += np.sum(output_delta, axis=0) * self.learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0) * self.learning_rate

    def train(self, X, y):
        for _ in range(self.epochs):
            output = self.feedforward(X)
            self.backpropagate(X, y, output)

# Training MLP
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])  # XOR operation
mlp = MLP(input_size=2, hidden_size=4, output_size=1)
mlp.train(X, y)

print("\nMLP Predictions (XOR Gate):")
for inputs in X:
    print(f"Input: {inputs}, Prediction: {mlp.feedforward(inputs)[0]:.3f}")

# 3. Hopfield Network
class HopfieldNetwork:
    def __init__(self, num_neurons):
        self.num_neurons = num_neurons
        self.weights = np.zeros((num_neurons, num_neurons))

    def train(self, patterns):
        # Train using Hebbian learning
        for p in patterns:
            self.weights += np.outer(p, p)
        np.fill_diagonal(self.weights, 0)

    def predict(self, pattern, steps=10):
        # Update neurons synchronously
        output = pattern.copy()
        for _ in range(steps):
            output = np.sign(self.weights.dot(output))
        return output

# Training Hopfield Network
patterns = np.array([[1, -1, 1, -1], [-1, 1, -1, 1]])  # Memorize two patterns
hopfield_net = HopfieldNetwork(num_neurons=4)
hopfield_net.train(patterns)

# Testing Hopfield Network with noisy input
test_pattern = np.array([1, -1, -1, -1])  # Slightly noisy pattern
output_pattern = hopfield_net.predict(test_pattern)

print("\nHopfield Network Prediction (Pattern Recovery):")
print(f"Test Pattern: {test_pattern}")
print(f"Recovered Pattern: {output_pattern}")


Perceptron Predictions (AND Gate):
Input: [0 0], Prediction: 0
Input: [0 1], Prediction: 0
Input: [1 0], Prediction: 0
Input: [1 1], Prediction: 1

MLP Predictions (XOR Gate):
Input: [0 0], Prediction: 0.035
Input: [0 1], Prediction: 0.960
Input: [1 0], Prediction: 0.958
Input: [1 1], Prediction: 0.044

Hopfield Network Prediction (Pattern Recovery):
Test Pattern: [ 1 -1 -1 -1]
Recovered Pattern: [ 1. -1.  1. -1.]
